# 📊 Evaluation Script

Let's start by reading the data

In [ ]:
# DO NOT CHANGE
import cv2
import numpy as np
import glob
import time

## 1. Fill x_test and y_test:
x_test = []
y_test = []
fonts = [ 'IBM Plex Sans Arabic', 'Lemonada', 'Marhey', 'Scheherazade New']

for font in fonts:
    for filename in sorted(glob.glob(f'/content/test/{font}/*.jpeg')):
        img = cv2.imread(filename, cv2.COLOR_BGR2RGB)
        x_test.append(img)
        y_test.append(fonts.index(font))

# 2. Convert them to Numpy arrays:
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print("shape of x_data:", x_test.shape, "shape of y_data:", y_test.shape)
assert set(y_test) == {0, 1, 2, 3}

Here you define your predict function following this specific interface:

In [ ]:
from skimage.filters import unsharp_mask

'''
This function takes an RGB image of dimensions (1181, 1181, 3) from the test set and returns integer prediction ∈ {0,1,2,3}
'''
import pickle

def remove_noise (image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    X = cv2.bilateralFilter(gray, 15, sigmaColor=10, sigmaSpace=10)
    median = cv2.medianBlur(X, 5)
    result_2 = unsharp_mask(median, radius=10, amount=4)*255
    result_2 = np.uint8(result_2)
    sharpen = cv2.Canny(result_2, 100,250)
    return sharpen

def predict_rf (descriptors, rf):
    predictions = np.zeros(4)
    for des in descriptors:
        pred = rf.predict([des])
        ind = np.int64(pred[0])
        predictions[ind]+=1
    return np.argmax(predictions)

sift = cv2.SIFT_create(500)

def make_prediction(x):
    # DEFINE YOUR FUNCTION HERE AND DO NOT CHANGE ANYTHING ELSE IN THE NOTEBOOK
    rf = pickle.load(open('rf.pk1' , 'rb'))
    removed_noise = remove_noise(x)
    kp , descriptors= sift.detectAndCompute(removed_noise,None)
    if descriptors is not None:
        predic = predict_rf(descriptors, rf)
        return predic
    else:
        return 0

# Fill your team number here
TEAM_NUM = "13"

Now let's compute the accuracy of the model:

In [ ]:
# DO NOT CHANGE
y_pred = []

start_time = time.time()
for x in x_test:
    assert x.shape == (1181, 1181, 3)
    ŷ = make_prediction(x)
    y_pred.append(ŷ)
end_time = time.time()

y_pred = np.asarray(y_pred)
accuracy = np.mean(y_pred == y_test)
total_time = end_time - start_time
print(f"Team {TEAM_NUM} got accuracy: {accuracy:.2%}")
print(f"Team {TEAM_NUM} got runtime: {total_time:.2%}")